In [ ]:
import pandas as pd
import numpy as np

# Importando os dados
Obs: é preciso carregar os arquivos no colab

In [ ]:
demografico = pd.read_csv('1_demografico.csv',sep=';')
demografico.head()

In [ ]:
renda_gastos = pd.read_csv('2_renda_gastos.csv',sep=';')
renda_gastos.head()

In [ ]:
bens = pd.read_csv('3_bens.csv',sep=';')
bens.head()

In [ ]:
demografico.shape, renda_gastos.shape, bens.shape

**Questão 1**


In [ ]:
df_dados = demografico.merge(renda_gastos, how='inner',left_on=['ID','Region'],right_on=['ID','Region'])

In [ ]:
df_dados.shape

In [ ]:
df_dados.head()

In [ ]:
df_dados = pd.concat([df_dados,bens], axis=1)

In [ ]:
df_dados.shape

In [ ]:
df_dados.dtypes

In [ ]:
df_dados

## Questão 2

In [ ]:
df_info = df_dados.describe()
df_info

# TODO: podemos usar o describe ou devemos usar groupby, etc?

## Questão 3

In [ ]:
# TODO: utilizar between para criar a mascara
def elimina_outliers(serie_original):
  serie = serie_original.copy()
  estatisticas = serie.describe()

  q1 = estatisticas['25%']
  q3 = estatisticas['75%']
  iqr = q3 - q1
  minimum = q1 - 1.5 * iqr
  maximum = q3 + 1.5 * iqr

  # mascara_minimum = serie < minimum
  # mascara_maximum = serie > maximum

  # serie[mascara_minimum] = np.NaN 
  # serie[mascara_maximum] = np.NaN 

  mascara = serie.between(minimum, maximum, inclusive = True)
  serie[~mascara] = np.NaN 

  return serie

In [ ]:
# Separando apenas os dados numáericos
df_dados_numericos = df_dados.select_dtypes(include=[np.number]) 

In [ ]:
df_dados_numericos2 = df_dados_numericos.copy()
for col in df_dados_numericos.columns:
  df_dados_numericos2[col] = elimina_outliers(df_dados_numericos[col])


In [ ]:
df_dados_numericos2.describe()

In [ ]:
# TODO: ver como funciona o apply 
# isso até parece funcionar, mas demora demais

df_dados_numericos3 = df_dados_numericos.copy()
df_dados_numericos3 = df_dados_numericos3.apply(elimina_outliers, axis=1,raw=False)

df_dados_numericos3.describe()


## Questão 4

In [ ]:
def transforma_outliers_em_mediana(serie_original):
  serie = serie_original.copy()
  estatisticas = serie.describe()

  q1 = estatisticas['25%']
  q3 = estatisticas['75%']
  iqr = q3 - q1
  minimum = q1 - 1.5 * iqr
  maximum = q3 + 1.5 * iqr
  mascara_minimum = serie < minimum
  mascara_maximum = serie > maximum

  serie[mascara_minimum] = serie.median() 
  serie[mascara_maximum] = serie.median()

  return serie

In [ ]:
# TODO: ver como funciona o apply
#df_dados_numericos.apply(elimina_outliers, axis=1)

df_dados_numericos3 = df_dados_numericos.copy()
for col in df_dados_numericos.columns:
  df_dados_numericos3[col] = transforma_outliers_em_mediana(df_dados_numericos[col])

In [ ]:
df_dados_numericos3.describe()

### Questão 5

In [ ]:
df_nao_numericos = df_dados.select_dtypes(exclude=[np.number]) 
df_nao_numericos.head()

In [ ]:
def relatorio_q5(df, col):
    freq_abs = df[col].value_counts()
    freq_abs.name = 'FreqAbs'
    freq_rel = round((df[col].value_counts()/len(df)),2)
    freq_rel.name = 'FreqRel'
    freq_relp = freq_rel*100
    freq_relp.name = 'FreqRel%'
    freq_rel_acum = round((df[col].value_counts()/len(df)).cumsum(),2)
    freq_rel_acum.name = 'FreqRelAcum'
    relatorio5 = pd.concat([freq_abs,freq_rel,freq_relp,freq_rel_acum ],axis=1)
    relatorio5.index.name = col
    relatorio5.loc["Total"] = relatorio5.sum()
    relatorio5.loc["Total"]['FreqRelAcum'] = 1
    return relatorio5

In [ ]:
for col in df_nao_numericos:
    display(relatorio_q5(df_nao_numericos, col))
    print()
    print()

## Questão 6

In [ ]:
quantil_90 = df_dados['Total Household Income'].quantile(0.9)
maiores_10 = df_dados['Total Household Income'] > quantil_90
df_maiores = df_dados[maiores_10]
df_maiores.to_csv('out_10_maiores_rendas.csv')

## Questão 7

### Exploração de dados

In [ ]:
import seaborn as sns

In [ ]:
# sns.boxplot(data=df_dados[df_dados['Total Household Income']<0.2e7], x='Total Household Income', y='Type of Building/House')
sns.histplot(x='Total Household Income', data=df_dados, kde=True, hue = 'Type of Building/House', log_scale=True)

In [ ]:
# sns.boxplot(data=df_dados[df_dados['Total Household Income']<0.2e7], x='Total Household Income', y='Household Head Marital Status')
sns.histplot(x='Total Household Income', data=df_dados, kde=True, hue = 'Household Head Marital Status', log_scale=True)

In [ ]:
# sns.boxplot(data=df_dados[df_dados['Total Household Income']<0.2e7], x='Total Household Income', y='Main Source of Income')
sns.histplot(x='Total Household Income', data=df_dados, kde=True, hue = 'Main Source of Income', log_scale=True)

In [ ]:
sns.histplot(x='Total Household Income', data=df_dados, kde=True, hue = 'Type of Household', log_scale=True)

In [ ]:
# sns.boxplot(data=df_dados[df_dados['Total Household Income']<0.2e7], x='Total Household Income', y='Household Head Sex')
sns.histplot(x='Total Household Income', data=df_dados, kde=True, hue = 'Household Head Sex', log_scale=True)

### Solução da questão

In [ ]:
df_dummies = pd.get_dummies(df_dados, columns= ['Household Head Sex','Type of Household','Main Source of Income'])
df_dummies.head()

## Questão 8

In [ ]:
df_nao_maiores = df_dados[~maiores_10]

In [ ]:
df_maiores_dummies = pd.get_dummies(df_maiores, columns= ['Household Head Sex','Type of Household','Main Source of Income'])
df_nao_maiores_dummies = pd.get_dummies(df_nao_maiores, columns= ['Household Head Sex','Type of Household','Main Source of Income'])

In [ ]:
media_maiores = df_maiores_dummies.mean()
media_nao_maiores = df_nao_maiores_dummies.mean()

In [ ]:
df_comparacao = pd.DataFrame({'maiores':media_maiores, 'nao_maiores':media_nao_maiores})
df_comparacao

In [ ]:
df_normalizado = df_comparacao['maiores']/df_comparacao['nao_maiores']-1

In [ ]:
df_normalizado

A população com mais renda:
- Tem casas maiores e com mais quartos
- Gasta mais em todas as categorias (que tem Expeditures no nome)
- Ganha mais em fonte de renda extra (Total Income from Entrepreneurial Acitivites)
- Mas tem menos casos onde empreendedorismo é a fonte principal de renda
- Tem mais mulheres liderando a família
